### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Load model

Goal: To adapt `DeepSeek-R1-0528-Qwen3-8B` into Facencial Customer services with reasoning capability via SFT & Distillation by using syntetic dataset & GPT-5 with as a teacher.

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
lora_rank = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-09 11:32:07 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-09 11:32:07 [__init__.py:239] Automatically detected platform cuda.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.4: Fast Qwen3 patching. Transformers: 4.55.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.78 GB. Also swap space = 0 GB.


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.8.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Reasoning chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [ ]:
reasoning_start = "<think>"
reasoning_end   = "<think>"
solution_start  = "<output>"
solution_end    = "</output>"

system_prompt = \
f"""Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada {reasoning_start} dan {reasoning_end}.
lalu, letakkan jawaban kamu di antara {solution_start}{solution_end}"""
system_prompt

'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output>'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [ ]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [ ]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>What is 1+1?<think>I think it's 2.<think><output>2</output><｜end▁of▁sentence｜>What is 2+2?<think>"

### Fine-tuning phase I
We now use a Dataset of Facencial [Facencial Dataset](https://huggingface.co/datasets/MMumtazSakho/facencial) which was developed by Sakho (me). There are only 30 rows of sample. Those sample will be added more as the model taught by gpt-5 RAG

```
# This is formatted as code
```



In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("MMumtazSakho/facencial-cot", split = "train")
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

dataset

,expected_answer,problem,generated_solution
0,Facencial menggunakan teknologi generatif AI t...,Apa keunggulan utama Facencial?,Mari kita pikirkan langkah demi langkah.\n1. P...
1,"Klik tombol 'GENERATE SEKARANG' untuk login, p...",Bagaimana cara mulai generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...
2,Realistic Lite memiliki kualitas basic dengan ...,Apa perbedaan Realistic Lite dan Ultra Realistic?,Mari kita pikirkan langkah demi langkah.\n1. P...
3,Paket termurah adalah Realistic Lite mulai dar...,Berapa harga paket termurah?,Mari kita pikirkan langkah demi langkah.\n1. P...
4,Proses generate foto memakan waktu 20 detik un...,Berapa lama proses generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...
5,Edisi Kampus memungkinkan Anda berfoto AI di l...,Apa itu Edisi Kampus?,Mari kita pikirkan langkah demi langkah.\n1. P...
6,Kami memahami kebutuhan Anda dan berupaya memb...,Apakah ada garansi kepuasan?,Mari kita pikirkan langkah demi langkah.\n1. P...
7,Paket 100 credits (Paket D) dijual seharga Rp5...,Berapa harga paket 100 credits?,Mari kita pikirkan langkah demi langkah.\n1. P...
8,"Ya, Facencial memiliki sistem AI validasi gamb...",Apakah ada validasi gambar sebelum upload?,Mari kita pikirkan langkah demi langkah.\n1. P...
9,Formal Pas Foto untuk kebutuhan profesional de...,Apa perbedaan Formal Pas Foto dan Foto Seni?,Mari kita pikirkan langkah demi langkah.\n1. P...


We have to format the dataset to follow our GRPO style formatting:

In [ ]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    thoughts = thoughts.strip()
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis= 1)

Check to see if it worked:

In [ ]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Apa keunggulan utama Facencial?<think>Mari kita pikirkan langkah demi langkah.\n1. Pertanyaannya tentang keunggulan utama Facencial.\n2. Facencial adalah layanan AI generatif untuk membuat foto.\n3. Keunggulannya ada pada teknologi AI canggih, hasil premium, cepat, dan tanpa perlu photoshoot fisik.<think><output>Facencial menggunakan teknologi generatif AI tercanggih untuk menghasilkan foto premium dalam hitungan detik tanpa perlu photoshoot fisik.</output><｜end▁of▁sentence｜>'

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [ ]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

(29, 5)

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [ ]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'Messages', 'N', 'text', '__index_level_0__'],
    num_rows: 29
})

In [ ]:
dataset["text"]

['Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Apa keunggulan utama Facencial?<think>Mari kita pikirkan langkah demi langkah.\n1. Pertanyaannya tentang keunggulan utama Facencial.\n2. Facencial adalah layanan AI generatif untuk membuat foto.\n3. Keunggulannya ada pada teknologi AI canggih, hasil premium, cepat, dan tanpa perlu photoshoot fisik.<think><output>Facencial menggunakan teknologi generatif AI tercanggih untuk menghasilkan foto premium dalam hitungan detik tanpa perlu photoshoot fisik.</output><｜end▁of▁sentence｜>',
 "Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Bagaimana cara mulai generate foto?<think>Mar

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 5, # I use 5 epochs full
        learning_rate = 2e-4,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 29 | Num Epochs = 5 | Total steps = 145
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)


Step,Training Loss
5,2.301800
10,1.712600
15,1.182100
20,0.968200
25,0.846300
30,0.903200
35,0.572500
40,0.564900
45,0.548000
50,0.573200


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=145, training_loss=0.46338953673839567, metrics={'train_runtime': 196.4819, 'train_samples_per_second': 0.738, 'train_steps_per_second': 0.738, 'total_flos': 1228068737003520.0, 'train_loss': 0.46338953673839567})

Let's check if the model has learnt to follow the custom format:

In [ ]:
text = tokenizer.apply_chat_template(
    [{'content': 'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output>',
  'role': 'system'},
 {'content': 'saya mau paket yang murah dan cepat. paket apa yang saya pilih?', 'role': 'user'}],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan <think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>saya mau paket yang murah dan cepat. paket apa yang saya pilih?<think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan paket yang murah dan cepat untuk layanan foto.
2. Dari daftar paket yang ada, Realistic Lite adalah yang termurah dengan harga Rp10.000 di awal.
3. Realistic Lite juga memiliki waktu proses tercepat yaitu sekitar 20 detik, sehingga pengguna dapat langsung mendapatkan hasilnya tanpa perlu menunggu lama.
4. Karena harga rendah dan waktu proses singkat, Realistic Lite menjadi pilihan terbaik untuk pengguna yang ingin hasil cepat dan hemat biaya.<think><output>Untuk paket yang murah dan cepat, kamu bisa memilih Realistic Lite dengan harga Rp10.000 di awal dan waktu proses hanya 20 detik.</output><｜end▁of▁sentence｜>


# Model evaluation Phase 1

In [14]:
from openai import OpenAI
client = OpenAI(api_key="YOUR-KEY")



In [ ]:
from datasets import load_dataset

dataset = load_dataset("MMumtazSakho/facencial-cot", split = "train")

## Create RAG Pipeline for evaluation

Evaluation process flow:
1. If finetuned model give wrong answer, gpt-5 RAG will create correct/suggestion answer based on dataset
2. the correction answer will stored to dataset for future training

In [ ]:

import pickle
from openai import OpenAI
from datasets import load_dataset

def get_embeddings(texts, model="text-embedding-3-large"):
    response = client.embeddings.create(
        model=model,
        input=texts
    )
    return [item.embedding for item in response.data]

# Buat embedding dan simpan ke list
data_with_embeddings = []
batch_size = 10
for i in range(0, len(dataset), batch_size):
    batch_texts = [
        dataset[i + j]["problem"] + " " + dataset[i + j]["generated_solution"]
        for j in range(min(batch_size, len(dataset) - i))
    ]
    batch_embeddings = get_embeddings(batch_texts)

    for j, emb in enumerate(batch_embeddings):
        item = {
            "problem": dataset[i + j]["problem"],
            "generated_solution": dataset[i + j]["generated_solution"],  # CoT
            "expected_answer": dataset[i + j]["expected_answer"],
            "embedding": emb
        }
        data_with_embeddings.append(item)

# Simpan ke file
with open("dataset_with_embeddings.pkl", "wb") as f:
    pickle.dump(data_with_embeddings, f)

print("✅ Embedding berhasil dibuat dan disimpan ke dataset_with_embeddings.pkl")

✅ Embedding berhasil dibuat dan disimpan ke dataset_with_embeddings.pkl


In [13]:
import json
from tqdm import tqdm

def evaluate_accuracy(test_queries):
    correct_count = 0
    total_count = len(test_queries)
    corrections_dataset = []

    for query in tqdm(test_queries, desc="Evaluating"):
        # 1. Jalankan model fine-tuned
        model_output = run_finetuned_model(query)

        # 2. Ambil retrieval dari RAG
        retrieved_docs = search(query, top_k=3)
        context = "\n\n".join(
            [f"Problem: {doc['problem']}\nCoT: {doc['generated_solution']}\nAnswer: {doc['expected_answer']}"
             for doc in retrieved_docs]
        )

        rag_prompt = f"""
          Kamu adalah AI evaluator yang bertugas menilai kualitas jawaban customer service Facencial.

          Informasi yang kamu miliki:
          1. Pertanyaan user:
          {query}

          2. Jawaban model (termasuk chain of thought jika ada):
          {model_output}

          3. Informasi relevan dari hasil retrieval (berisi Problem, CoT, dan Answer):
          {context}

          Instruksi evaluasi:
          1. Bandingkan jawaban model dengan informasi retrieval untuk menentukan apakah jawabannya BENAR atau SALAH.
          2. Periksa chain of thought (proses berpikir) pada jawaban model:
            - Jika ada kesalahan logika atau ketidaksesuaian fatal terhadap informasi retrieval, anggap jawaban SALAH.
          3. Jika jawaban SALAH:
            - Buat perbaikan jawaban yang BENAR berdasarkan informasi retrieval saja.
            - Gunakan format:
              Mari kita pikirkan langkah demi langkah. [jelaskan alasan, analisis, dan sebab-akibat secara obyektif tanpa menyebut "jawaban model sebelumnya"]
              <output>[jawaban akhir yang benar]</output>
          4. Jika pertanyaan user berada di luar konteks atau domain Facencial, anggap SALAH dan buat improved_answer sesuai format pada poin 3.
          5. Jangan memberikan komentar atau kritik langsung terhadap jawaban model; fokus hanya pada analisis dan perbaikan berdasarkan retrieval.
          6. Hasil akhir evaluasi harus dalam format JSON valid:
          {{
            "evaluation": "BENAR" atau "SALAH",
            "improved_answer": "Jika SALAH, isi perbaikan jawaban; jika BENAR, isi string kosong"
          }}
          """
        response = client.responses.create(
            model="gpt-5-mini",
            input=rag_prompt
        )

        try:
            eval_result = json.loads(response.output_text)
        except json.JSONDecodeError:
            print(f"[Warning] JSON invalid untuk query: {query}")
            continue

        if eval_result.get("evaluation", "").upper() == "BENAR":
            correct_count += 1
        else:
            corrections_dataset.append({
                "instruction": query,
                "output": eval_result.get("improved_answer", "")
            })

    accuracy = correct_count / total_count if total_count > 0 else 0
    print(f"\n✅ Akurasi: {accuracy:.2%} ({correct_count}/{total_count} benar)")

    return accuracy, corrections_dataset



In [15]:
# Rag pipeline 2
from transformers import TextStreamer
from openai import OpenAI
import pickle

with open("dataset_with_embeddings.pkl", "rb") as f:
    data_with_embeddings = pickle.load(f)

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_embeddings(texts, model="text-embedding-3-large"):
    response = client.embeddings.create(model=model, input=texts)
    return [item.embedding for item in response.data]

def search(query, top_k=3):
    query_emb = get_embeddings([query])[0]
    embeddings_matrix = np.array([item["embedding"] for item in data_with_embeddings])
    similarities = cosine_similarity([query_emb], embeddings_matrix)[0]
    top_idx = np.argsort(similarities)[::-1][:top_k]
    return [data_with_embeddings[i] for i in top_idx]

def run_finetuned_model(user_input):
    text = tokenizer.apply_chat_template(
        [
            {
                'content': 'Kamu adalah chatbot costumer service Facencial.\n'
                           'pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\n'
                           'letakkan pikiran kamu pada <think> dan </think>.\n'
                           'lalu, letakkan jawaban kamu di antara <output></output>',
                'role': 'system'
            },
            {'content': user_input, 'role': 'user'}
        ],
        tokenize=False,
        add_generation_prompt=True
    )
    # Generate output
    streamer = TextStreamer(tokenizer, skip_prompt=False)
    outputs = model.generate(
        **tokenizer(text, return_tensors="pt").to("cuda"),
        temperature=0,
        max_new_tokens=1024,
        streamer=streamer
    )
    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# ==== 5. Fungsi Evaluasi dengan RAG ====
def rag_evaluate(user_input, model_output):
    retrieved_docs = search(user_input, top_k=3)

    # Gabungkan retrieval jadi context
    context = "\n\n".join(
        [f"Problem: {doc['problem']}\nCoT: {doc['generated_solution']}\nAnswer: {doc['expected_answer']}"
         for doc in retrieved_docs]
    )

    # Prompt untuk RAG memberikan penilaian
    rag_prompt = f"""
Kamu adalah AI evaluator.
Pertanyaan user: {user_input}

Jawaban model:
{model_output}

Informasi relevan dari retrieval:
{context}

Tugas:
1. Tentukan apakah jawaban model BENAR atau SALAH berdasarkan informasi retrieval.
2. Jika SALAH, buat perbaikan jawaban yang benar.
3. Format output dalam JSON seperti ini:
{{
  "evaluation": "BENAR" atau "SALAH",
  "improved_answer": "jika salah, isi perbaikan; jika benar, kosong"
}}
"""

    response = client.responses.create(
        model="gpt-5-mini",
        input=rag_prompt
    )
    return response.output_text




In [ ]:
# see if the function works properly
accuracy, corrections = evaluate_accuracy(['apakah jumlah credit bisa custom?'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<｜begin▁of▁sentence｜>Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜User｜>apakah jumlah credit bisa custom?<｜Assistant｜><think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan apakah jumlah credit bisa diatur sesuai keinginan pengguna.
2. Sistem Facencial berbasis credits karena setiap kali AI generate gambar, komputasi yang dibutuhkan cukup besar.
3. Untuk menjaga kualitas hasil dan memastikan biaya operasional tetap tercover, jumlah credit dibatasi pada 100 per paket.
4. Batasan ini juga berfungsi sebagai pelindung pengguna agar tidak terjadi hasil yang kurang memuaskan akibat penggunaan credit secara berlebihan dalam satu kali paket.
5. Akibatnya, meskipun pengguna mungkin ingin menggunakan jumlah credit yang berbeda, sistem tidak mengizinkan customasi jumlah credit di luar 100 credits.<think><output>

Evaluating: 100%|██████████| 1/1 [00:39<00:00, 39.92s/it]


✅ Akurasi: 100.00% (1/1 benar)


In [ ]:
corrections

[{'instruction': 'apakah jumlah credit bisa custom?',
  'output': 'Mari kita pikirkan langkah demi langkah.\n1. Informasi yang tersedia menyebutkan paket kredit dengan jumlah tetap, misalnya paket Realistic Lite (16 credits) dan paket 100 credits (Paket D).\n2. Disebutkan juga bahwa pembelian paket yang lebih besar (hingga 100 credits) memberikan harga per foto yang lebih murah.\n3. Dalam informasi retrieval tidak ada keterangan yang menyatakan adanya opsi untuk meng-custom jumlah credit di luar paket yang disediakan.\n4. Karena tidak ada bukti bahwa customisasi jumlah credit didukung, kita tidak bisa memastikan adanya fitur tersebut — kemungkinan besar pengguna harus memilih dari paket yang tersedia.\n5. Jika pengguna membutuhkan jumlah khusus di luar paket yang tercantum, langkah yang disarankan adalah menghubungi layanan pelanggan untuk menanyakan kemungkinan solusi khusus.\n\n<output>Berdasarkan info yang ada, Facencial menawarkan paket kredit dengan jumlah tetap (contoh: 16 credit

## Sample test case

In [ ]:

test_case = ['Apakah harga per foto di Ultra Realistic lebih mahal daripada Lite?',
 'Apa syarat foto yang bisa diupload ke Facencial Ultra?',
 'Mengapa Facencial Ultra lebih lama prosesnya daripada Lite?',
 'Mana yang lebih cocok untuk dokumen resmi: Formal Pas Foto atau Foto Seni?',
 'Apa yang harus dilakukan jika proses training Ultra gagal?',
 'Facencial Ultra lebih bagus dari Lite, ya?',
 'Credit habis tapi mau generate ulang, gimana?',
 'Berapa harga Paket D?',
 'Berapa harga Paket A?',
 'Ada berapa foto yang saya dapatkan di paket D?',
 'Ada berapa foto yang saya dapatkan di paket B?',
 'Apakah biaya pelatihan AI berlaku untuk semua paket?',
 'Paket mana yang paling hemat untuk kebutuhan foto banyak?',
 'Berapa total biaya awal untuk Facencial Ultra termasuk pelatihan AI?',
 'Kalau wajahnya pakai kacamata, apakah bisa terdeteksi?',
 'saya sudah pernah membeli paket ultra. tetapi credit sudah habis. apakah harus membayar biaya pelatihan lagi?',
 'berapa lama prosesnya?',
 'apakah jumlah credit bisa custom?',
 'siapa nama presiden indonesia?',
 'berapa nilai 1+1?',
 'bagaimana cara memasak?',
 'Berapa lama model AI saya tersimpan? apakah saya bisa menghapusnya? jika terhapus, maka apa akibatnya?']

In [ ]:
accuracy, corrections = evaluate_accuracy(test_case)

Evaluating:   0%|          | 0/22 [00:00<?, ?it/s]

<｜begin▁of▁sentence｜>Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜User｜>Apakah harga per foto di Ultra Realistic lebih mahal daripada Lite?<｜Assistant｜><think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan perbedaan harga antara paket Ultra Realistic dan Lite di layanan Facencial.
2. Berdasarkan struktur paket, Ultra Realistic memerlukan biaya pelatihan AI terlebih dahulu sebesar Rp16.000 di awal proses.
3. Setelah pelatihan AI selesai, pengguna dapat mengenerate foto sebanyak mungkin dengan menggunakan credit per foto.
4. Sedangkan paket Lite tidak melewati proses pelatihan AI sehingga tidak dikenai biaya tambahan di awal.
5. Akibatnya, harga per foto di Ultra Realistic menjadi lebih mahal karena adanya biaya pelatihan AI sebesar Rp16.000 yang dibagi di antara harga total paket.<think><output>Harga 

Evaluating:   5%|▍         | 1/22 [00:27<09:27, 27.03s/it]

<｜begin▁of▁sentence｜>Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜User｜>Apa syarat foto yang bisa diupload ke Facencial Ultra?<｜Assistant｜><think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan syarat foto yang bisa diupload ke Facencial Ultra.
2. Facencial Ultra memerlukan kualitas foto yang tinggi untuk menghasilkan gambar yang realistis.
3. Foto harus memiliki posisi, ukuran, dan resolusi yang sesuai agar proses analisis dan generate bisa berjalan dengan baik.
4. Kualitas foto yang baik akan memungkinkan AI mengenali fitur wajah dan detail dengan akurat.
5. Akibatnya, hasil generate akan lebih akurat dan realistis.<think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan syarat foto yang bisa diupload ke Facencial Ultra.
2. Facencial Ultra memerlukan kualitas foto yang tinggi untuk m

Evaluating:   9%|▉         | 2/22 [02:06<23:09, 69.50s/it]

...

<｜begin▁of▁sentence｜>Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜User｜>bagaimana cara memasak?<｜Assistant｜><think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan prosedur memasak produk Facencial.
2. Produk ini berbentuk lembaran sehingga dapat digunakan langsung sebagai bahan dasar foto.
3. Proses memasak tidak diperlukan karena produk ini sudah siap pakai.
4. Pengguna cukup membuka lembaran, mengatur posisi, dan mengambil foto di depan latar belakang yang tersedia.<think><output>Facencial tidak memerlukan proses memasak. Kamu dapat menggunakan produk ini langsung dengan cara membuka lembaran, mengatur posisi, dan mengambil foto di depan latar belakkan yang tersedia.</output><｜end▁of▁sentence｜>


Evaluating:  95%|█████████▌| 21/22 [12:03<00:27, 27.42s/it]

<｜begin▁of▁sentence｜>Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜User｜>Berapa lama model AI saya tersimpan? apakah saya bisa menghapusnya? jika terhapus, maka apa akibatnya?<｜Assistant｜><think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan durasi penyimpanan model AI, kemampuan penghapusan, dan akibatnya jika terhapus.
2. Model AI biasanya disimpan untuk sesi pengguna agar dapat diproses kembali tanpa perlu pelatihan ulang.
3. Durasi penyimpanan tergantung pada kebijakan layanan, biasanya beberapa menit hingga jam, untuk memastikan kualitas hasil.
4. Pengguna dapat menghapus model AI kapan saja melalui antarmuka pengaturan.
5. Jika model terhapus, pengguna harus memulai proses pelatihan ulang dari awal untuk menghasilkan model baru, yang memerlukan biaya komputasi tambahan dan waktu.
6. Akibatnya, j

Evaluating: 100%|██████████| 22/22 [13:45<00:00, 37.52s/it]


✅ Akurasi: 36.36% (8/22 benar)


## Accuracy Phase I result: 36%

Next we will train the model again based on correction answer that taught by gpt-5 RAG

In [ ]:
import re

def convert_corrections_format(corrections_dataset):
    converted = []
    for item in corrections_dataset:
        problem = item["instruction"]
        output_text = item["output"]

        # Pisahkan CoT (generated_solution) dan expected_answer dari <output>...</output>
        match = re.search(r"(.*)<output>(.*?)</output>", output_text, re.DOTALL)
        if match:
            generated_solution = match.group(1).strip()
            expected_answer = match.group(2).strip()
        else:
            generated_solution = output_text.strip()
            expected_answer = ""

        converted.append({
            "problem": problem,
            "generated_solution": generated_solution,
            "expected_answer": expected_answer
        })

    return converted

In [ ]:
new_dataset = convert_corrections_format(corrections_dataset)


In [ ]:
new_dataset

[{'problem': 'Apakah harga per foto di Ultra Realistic lebih mahal daripada Lite?',
  'generated_solution': 'Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah harga per foto Ultra Realistic lebih mahal daripada Lite. Dari informasi yang tersedia, perbedaan yang disebut adalah adanya biaya pelatihan AI Rp16.000 sekali di awal untuk Ultra Realistic, sedangkan Lite tidak memiliki biaya ini. Tidak ada informasi yang menyatakan harga per foto (credit) Ultra lebih mahal daripada Lite, sehingga kesimpulan tersebut tidak bisa dipastikan dari data ini.',
  'expected_answer': 'Tidak ada informasi bahwa harga per foto Ultra Realistic lebih mahal daripada Lite. Yang pasti, Ultra Realistic memiliki biaya pelatihan AI Rp16.000 sekali di awal, sedangkan Lite tidak. Untuk detail harga per foto/credit, silakan cek halaman harga atau hubungi kami.'},
 {'problem': 'Apa syarat foto yang bisa diupload ke Facencial Ultra?',
  'generated_solution': 'Mari kita pikirkan langkah demi langkah

In [ ]:
dataset.to_pandas()

,problem,generated_solution,expected_answer
0,Apa keunggulan utama Facencial?,Mari kita pikirkan langkah demi langkah.\n1. P...,Facencial menggunakan teknologi generatif AI t...
1,Bagaimana cara mulai generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...,"Klik tombol 'GENERATE SEKARANG' untuk login, p..."
2,Apa perbedaan Realistic Lite dan Ultra Realistic?,Mari kita pikirkan langkah demi langkah.\n1. P...,Realistic Lite memiliki kualitas basic dengan ...
3,Berapa harga paket termurah?,Mari kita pikirkan langkah demi langkah.\n1. P...,Paket termurah adalah Realistic Lite mulai dar...
4,Berapa lama proses generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...,Proses generate foto memakan waktu 20 detik un...
5,Apa itu Edisi Kampus?,Mari kita pikirkan langkah demi langkah.\n1. P...,Edisi Kampus memungkinkan Anda berfoto AI di l...
6,Apakah ada garansi kepuasan?,Mari kita pikirkan langkah demi langkah.\n1. P...,Kami memahami kebutuhan Anda dan berupaya memb...
7,Berapa harga paket 100 credits?,Mari kita pikirkan langkah demi langkah.\n1. P...,Paket 100 credits (Paket D) dijual seharga Rp5...
8,Apakah ada validasi gambar sebelum upload?,Mari kita pikirkan langkah demi langkah.\n1. P...,"Ya, Facencial memiliki sistem AI validasi gamb..."
9,Apa perbedaan Formal Pas Foto dan Foto Seni?,Mari kita pikirkan langkah demi langkah.\n1. P...,Formal Pas Foto untuk kebutuhan profesional de...


In [ ]:
import pandas as pd

df = pd.concat([dataset.to_pandas(), pd.DataFrame(new_dataset)])
df

,problem,generated_solution,expected_answer
0,Apa keunggulan utama Facencial?,Mari kita pikirkan langkah demi langkah.\n1. P...,Facencial menggunakan teknologi generatif AI t...
1,Bagaimana cara mulai generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...,"Klik tombol 'GENERATE SEKARANG' untuk login, p..."
2,Apa perbedaan Realistic Lite dan Ultra Realistic?,Mari kita pikirkan langkah demi langkah.\n1. P...,Realistic Lite memiliki kualitas basic dengan ...
3,Berapa harga paket termurah?,Mari kita pikirkan langkah demi langkah.\n1. P...,Paket termurah adalah Realistic Lite mulai dar...
4,Berapa lama proses generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...,Proses generate foto memakan waktu 20 detik un...
5,Apa itu Edisi Kampus?,Mari kita pikirkan langkah demi langkah.\n1. P...,Edisi Kampus memungkinkan Anda berfoto AI di l...
6,Apakah ada garansi kepuasan?,Mari kita pikirkan langkah demi langkah.\n1. P...,Kami memahami kebutuhan Anda dan berupaya memb...
7,Berapa harga paket 100 credits?,Mari kita pikirkan langkah demi langkah.\n1. P...,Paket 100 credits (Paket D) dijual seharga Rp5...
8,Apakah ada validasi gambar sebelum upload?,Mari kita pikirkan langkah demi langkah.\n1. P...,"Ya, Facencial memiliki sistem AI validasi gamb..."
9,Apa perbedaan Formal Pas Foto dan Foto Seni?,Mari kita pikirkan langkah demi langkah.\n1. P...,Formal Pas Foto untuk kebutuhan profesional de...


In [ ]:
df.to_json("dataset.json", orient="records", force_ascii=False, indent=2)
json_str = df.to_json(orient="records", force_ascii=False, indent=2)
print(json_str)

[
  {
    "problem":"Apa keunggulan utama Facencial?",
    "generated_solution":"Mari kita pikirkan langkah demi langkah.\n1. Pertanyaannya tentang keunggulan utama Facencial.\n2. Facencial adalah layanan AI generatif untuk membuat foto.\n3. Keunggulannya ada pada teknologi AI canggih, hasil premium, cepat, dan tanpa perlu photoshoot fisik.",
    "expected_answer":"Facencial menggunakan teknologi generatif AI tercanggih untuk menghasilkan foto premium dalam hitungan detik tanpa perlu photoshoot fisik."
  },
  {
    "problem":"Bagaimana cara mulai generate foto?",
    "generated_solution":"Mari kita pikirkan langkah demi langkah.\n1. Pertanyaan menanyakan prosedur untuk mulai membuat foto.\n2. Umumnya, pengguna harus login terlebih dahulu.\n3. Setelah login, mereka memilih paket layanan yang sesuai.\n4. Kemudian, mereka mengunggah foto dan memilih gaya yang diinginkan.",
    "expected_answer":"Klik tombol 'GENERATE SEKARANG' untuk login, pilih paket facencial, lalu ikuti proses upload f

# Fine-tuning phase II

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
lora_rank = 16

# Load the last model that we had finetuned
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "MMumtazSakho/Facencial-CoT-Lora",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-09 17:30:33 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-09 17:30:33 [__init__.py:239] Automatically detected platform cuda.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.4: Fast Qwen3 patching. Transformers: 4.55.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.78 GB. Also swap space = 0 GB.


...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/175M [00:00<?, ?B/s]

Unsloth 2025.8.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
reasoning_start = "<think>"
reasoning_end   = "<think>"
solution_start  = "<output>"
solution_end    = "</output>"

system_prompt = \
f"""Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada {reasoning_start} dan {reasoning_end}.
lalu, letakkan jawaban kamu di antara {solution_start}{solution_end}"""
system_prompt

'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output>'

In [ ]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset_updated = load_dataset("MMumtazSakho/facencial-cot-updated", split = "train")
dataset_updated = dataset_updated.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

dataset_updated

dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/45 [00:00<?, ? examples/s]

,expected_answer,problem,generated_solution
0,Facencial menggunakan teknologi generatif AI t...,Apa keunggulan utama Facencial?,Mari kita pikirkan langkah demi langkah.\n1. P...
1,"Klik tombol 'GENERATE SEKARANG' untuk login, p...",Bagaimana cara mulai generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...
2,Realistic Lite memiliki kualitas basic dengan ...,Apa perbedaan Realistic Lite dan Ultra Realistic?,Mari kita pikirkan langkah demi langkah.\n1. P...
3,Paket termurah adalah Realistic Lite mulai dar...,Berapa harga paket termurah?,Mari kita pikirkan langkah demi langkah.\n1. P...
4,Proses generate foto memakan waktu 20 detik un...,Berapa lama proses generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...
5,Edisi Kampus memungkinkan Anda berfoto AI di l...,Apa itu Edisi Kampus?,Mari kita pikirkan langkah demi langkah.\n1. P...
6,Kami memahami kebutuhan Anda dan berupaya memb...,Apakah ada garansi kepuasan?,Mari kita pikirkan langkah demi langkah.\n1. P...
7,Paket 100 credits (Paket D) dijual seharga Rp5...,Berapa harga paket 100 credits?,Mari kita pikirkan langkah demi langkah.\n1. P...
8,"Ya, Facencial memiliki sistem AI validasi gamb...",Apakah ada validasi gambar sebelum upload?,Mari kita pikirkan langkah demi langkah.\n1. P...
9,Formal Pas Foto untuk kebutuhan profesional de...,Apa perbedaan Formal Pas Foto dan Foto Seni?,Mari kita pikirkan langkah demi langkah.\n1. P...


In [ ]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    thoughts = thoughts.strip()
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset_updated["Messages"] = dataset_updated.apply(format_dataset, axis = 1)

In [ ]:
tokenizer.apply_chat_template(dataset_updated["Messages"][44], tokenize = False)

'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Berapa lama model AI saya tersimpan? apakah saya bisa menghapusnya? jika terhapus, maka apa akibatnya?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan mencakup tiga hal: durasi penyimpanan, opsi penghapusan, dan konsekuensi jika dihapus. Dari informasi yang tersedia, yang pasti adalah: model dapat dihapus lewat Pengaturan Akun, dan bila dihapus lalu Anda ingin menggunakan Ultra Realistic lagi, sistem harus melatih ulang dari awal dan biaya pelatihan Rp16.000 akan dikenakan kembali. Durasi penyimpanan spesifik tidak tercantum pada informasi yang saya miliki, sehingga sebaiknya merujuk ke kebijakan/halaman bantuan resmi untuk detail terbarunya.<think><output>• Durasi penyimpanan: Tidak tercantum pada info yang saya miliki. Silakan cek Pusat Bantua

In [ ]:
dataset_updated["N"] = dataset_updated["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset_updated = dataset_updated.loc[dataset_updated["N"] <= max_seq_length/2].copy()
dataset_updated.shape

(45, 5)

In [ ]:
from datasets import Dataset

dataset_updated["text"] = tokenizer.apply_chat_template(dataset_updated["Messages"].values.tolist(), tokenize = False)
dataset_updated = Dataset.from_pandas(dataset_updated)
dataset_updated

Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'Messages', 'N', 'text', '__index_level_0__'],
    num_rows: 45
})

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_updated,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/45 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 45 | Num Epochs = 5 | Total steps = 225
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)


Step,Training Loss
5,0.726800
10,0.919700
15,0.669300
20,0.180600
25,0.097200
30,0.569500
35,0.471800
40,0.359800
45,0.234100
50,0.344300


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=225, training_loss=0.17152857664558624, metrics={'train_runtime': 322.0462, 'train_samples_per_second': 0.699, 'train_steps_per_second': 0.699, 'total_flos': 1834926259660800.0, 'train_loss': 0.17152857664558624})

# Model Evaluation Phase II

In [ ]:
text = tokenizer.apply_chat_template(
    [{'content': 'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output>',
  'role': 'system'},
 {'content': 'Credit habis tapi mau generate ulang, gimana?', 'role': 'user'}],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan <think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Credit habis tapi mau generate ulang, gimana?<think>Mari kita pikirkan langkah demi langkah. Facencial menggunakan sistem credits; setiap kali generate foto, credits akan terpakai. Jika credits habis, Anda tidak perlu menunggu hari berikutnya—solusinya adalah menambah (membeli) credits. Setelah pembelian berhasil dan credits masuk, Anda bisa generate ulang sebanyak yang dibutuhkan selama credits mencukupi.<think><output>Credits Anda habis. Untuk generate ulang, silakan beli credits terlebih dulu: buka halaman pricing, pilih paket, klik "BELI SEKARANG", dan selesaikan pembayaran. Setelah credits masuk, Anda bisa mengulang generate foto kapan saja selama credits mencukupi. Jika pembayaran sudah berhasil tapi credits belum bertambah, hubungi kami.</output><｜end▁

In [ ]:
accuracy, corrections = evaluate_accuracy(test_case)

Evaluating:   0%|          | 0/22 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Apakah harga per foto di Ultra Realistic lebih mahal daripada Lite?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah harga per foto Ultra Realistic lebih mahal daripada Lite. Dari informasi yang tersedia, perbedaan yang disebut adalah adanya biaya pelatihan AI Rp16.000 sekali di awal untuk Ultra Realistic, sedangkan Lite tidak memiliki biaya ini. Tidak ada informasi yang menyatakan harga per foto (credit) Ultra lebih mahal daripada Lite, sehingga kesimpulan tersebut tidak bisa dipastikan dari data ini.<think><output>Tidak ada informasi bahwa harga per foto Ultra Realistic lebih mahal daripada Lite. Yang pasti, Ultra Realistic memiliki biaya pelatihan AI Rp16.000 sekali di awal, sedangkan Lite tidak. Untuk detail harga per foto/cred

Evaluating:   5%|▍         | 1/22 [00:42<14:51, 42.47s/it]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Apa syarat foto yang bisa diupload ke Facencial Ultra?<think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan syarat foto yang bisa diupload ke Facencial Ultra.
2. Facencial Ultra memerlukan foto dengan kualitas yang tinggi untuk menghasilkan gambar yang realistis.
3. Foto harus memiliki latar belakang polos (tidak berisik) agar proses ekstraksi gambar objeknya lebih akurat.
4. Resolusi foto juga penting; yang paling ideal adalah foto dengan ukuran minimal 1024x1024 piksel.
5. Kualitas foto yang baik akan meminimalisir kesalahan dalam proses training model, sehingga hasil akhirnya lebih memuaskan.<think><output>Facencial Ultra memerlukan foto dengan kualitas yang tinggi. Pastikan latar belakang polos (tidak berisik), dan resolusi minimal 102

Evaluating:   9%|▉         | 2/22 [01:17<12:43, 38.15s/it]

...

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>bagaimana cara memasak?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan pengguna tentang memasak tidak terkait dengan domain Facencial (platform AI untuk membuat/generate foto).  Berdasarkan dokumen, cara memulai Facencial adalah login, pilih paket, unggah foto, lalu pilih gaya; Facencial Lite membutuhkan 1 foto untuk analisis, sementara Facencial Ultra membutuhkan minimal 5 foto dengan verifikasi kualitas dan proses training ~5 menit sebelum bisa generate.<think><output>Facencial tidak terkait dengan proses memasak. Untuk mulai menggunakan Facencial:
- Klik tombol 'GENERATE SEKARANG' untuk login.
- Pilih paket Facencial (Lite atau Ultra).
- Jika memilih Lite: upload 1 foto wajah → sistem menganalisis gender dan atribut → pilih template/gaya → gene

Evaluating:  95%|█████████▌| 21/22 [08:45<00:24, 24.35s/it]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Berapa lama model AI saya tersimpan? apakah saya bisa menghapusnya? jika terhapus, maka apa akibatnya?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan mencakup tiga hal: durasi penyimpanan, opsi penghapusan, dan konsekuensi jika dihapus. Dari informasi yang tersedia, yang pasti adalah: model dapat dihapus lewat Pengaturan Akun, dan bila dihapus lalu Anda ingin menggunakan Ultra Realistic lagi, sistem harus melatih ulang dari awal dan biaya pelatihan Rp16.000 akan dikenakan kembali. Durasi penyimpanan spesifik tidak tercantum pada informasi yang saya miliki, sehingga sebaiknya merujuk ke kebijakan/halaman bantuan resmi untuk detail terbarunya.<think><output>• Durasi penyimpanan: Tidak tercantum pada info yang saya miliki. Silakan cek Pusat Bantuan a

Evaluating: 100%|██████████| 22/22 [09:16<00:00, 25.30s/it]


✅ Akurasi: 68.18% (15/22 benar)


## Accuracy Phase II result: 68%

We can see, as the data being updated, the model accuracy will increase

In [ ]:
tokenizer.push_to_hub("MMumtazSakho/Facencial-CoT-Lora-updated", token = "hf-token")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpg0owx1qf/tokenizer.json       :   0%|          | 29.0kB / 11.4MB            

In [ ]:
model.push_to_hub("MMumtazSakho/Facencial-CoT-Lora-updated", token = "hf-token")

README.md: 0.00B [00:00, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pmgbm3hip/adapter_model.safetensors:   0%|          |  558kB /  175MB            

Saved model to https://huggingface.co/MMumtazSakho/Facencial-CoT-Lora-updated


# Load model inference


In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "MMumtazSakho/Facencial-CoT-Lora",
    max_seq_length = 2048,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = 16,
    gpu_memory_utilization = 0.7,
)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.4: Fast Qwen3 patching. Transformers: 4.55.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.78 GB. Also swap space = 0 GB.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/175M [00:00<?, ?B/s]

Unsloth 2025.8.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
text = tokenizer.apply_chat_template(
    [{'content': 'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output>',
  'role': 'system'},
 {'content': 'saya frustasi dengan hasil foto yang tidak konsisten', 'role': 'user'}],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan <think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>saya frustasi dengan hasil foto yang tidak konsisten<think>Mari kita pikirkan langkah demi langkah.
1. Pengguna merasa frustasi karena hasil foto tidak konsisten sepanjang proses generate.
2. Ketidakkonsistenan ini bisa disebabkan oleh variasi gaya, latar belakang, atau komposisi yang tidak terkontrol.
3. Untuk mencegahnya, kami telah menyediakan tombol 'SKIP' pada halaman hasil foto.
4. Tombol 'SKIP' memungkinkan pengguna langsung mengulang generate tanpa harus melewati proses upload ulang atau verifikasi.
5. Dengan mengklik 'SKIP', pengguna dapat langsung memulai ulang proses generate dan memperoleh hasil yang lebih sesuai ekspektasi.
6. Oleh karena itu, rekomendasi terbaik adalah menggunakan tombel 'SKIP' jika pengguna kurang puas dengan hasil foto, untuk 

In [ ]:
evaluate_accuracy(['saya frustasi dengan hasil foto yang tidak konsisten'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>saya frustasi dengan hasil foto yang tidak konsisten<think>Mari kita pikirkan langkah demi langkah.
1. Pengguna merasa frustasi karena hasil foto tidak konsisten sepanjang proses generate.
2. Ketidakkonsistenan ini bisa disebabkan oleh variasi gaya, latar belakang, atau komposisi yang tidak terkontrol.
3. Untuk mencegahnya, kami telah menyediakan tombol 'SKIP' pada halaman hasil foto.
4. Tombol 'SKIP' memungkinkan pengguna langsung mengulang generate tanpa harus melewati proses upload ulang atau verifikasi.
5. Dengan mengklik 'SKIP', pengguna dapat langsung memulai ulang proses generate dan memperoleh hasil yang lebih sesuai ekspektasi.
6. Oleh karena itu, rekomendasi terbaik adalah menggunakan tombel 'SKIP' jika pengguna ingin mengubah gaya foto.
7. Akibatn

Evaluating: 100%|██████████| 1/1 [00:49<00:00, 49.15s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'saya frustasi dengan hasil foto yang tidak konsisten',
   'output': 'Mari kita pikirkan langkah demi langkah. 1. Pengguna merasa frustasi karena hasil foto yang tidak konsisten. 2. Ketidakkonsistenan biasanya disebabkan oleh kualitas foto input (pencahayaan, resolusi, ekspresi), serta variasi gaya yang dipilih. 3. Jika Anda menggunakan Facencial Lite, beberapa orang mungkin mendapatkan hasil yang kurang mirip karena tidak ada pelatihan personal; Facencial Ultra melakukan pelatihan AI terlebih dahulu sehingga hasil cenderung lebih konsisten dan menyerupai, dengan biaya pelatihan Rp16.000 di awal lalu bisa generate menggunakan credit. 4. Anda dapat mengulang proses generate selama masih memiliki credits, jadi Anda bisa mencoba ulang sambil menyesuaikan input atau gaya. 5. Facencial tidak menyediakan refund karena biaya komputasi AI yang tinggi, tetapi tim support dapat membantu menganalisis penyebab dan memberi panduan perbaikan. 6. Rekomendasi praktis berdasarka

In [ ]:
evaluate_accuracy(['adakah kontak whatsapp yang dapat dihubungi? saya mau bertanya'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>adakah kontak whatsapp yang dapat dihubungi? saya mau bertanya<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah ada kontak WhatsApp yang dapat dihubungi. Informasi ini biasanya tersedia pada halaman kontak atau bagian bantuan situs.<think><output>Kontak WhatsApp: +628123456789 (untuk pertanyaan umum) dan +628987654321 (untuk kerjasama). Hubungi kami Senin-Jumat pukul 09.00-17.00 WIB.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 1/1 [00:26<00:00, 26.09s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'adakah kontak whatsapp yang dapat dihubungi? saya mau bertanya',
   'output': 'Mari kita pikirkan langkah demi langkah. 1. Pengguna menanyakan apakah ada kontak WhatsApp yang dapat dihubungi. 2. Informasi yang tersedia menunjukkan bahwa dukungan Facencial disediakan melalui email dan tidak menyebutkan nomor WhatsApp. 3. Oleh karena itu, cara resmi untuk menghubungi tim support adalah melalui email agar tim dapat membantu memberikan solusi atau panduan.\n<output>Saat ini kami tidak menyediakan kontak WhatsApp. Untuk pertanyaan, silakan hubungi tim support kami melalui support@facencial.com agar tim kami dapat membantu dan memberikan panduan atau solusi yang sesuai.</output>'}])

In [ ]:
evaluate_accuracy(['jika saya membeli paket ultra, lalu kredit sudah habis. apakah saya harus membayar biaya training AI lagi? lalu bagaimana caranya supaya bisa generate foto lagi?'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>jika saya membeli paket ultra, lalu kredit sudah habis. apakah saya harus membayar biaya training AI lagi? lalu bagaimana caranya supaya bisa generate foto lagi?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan pengguna: jika membeli paket Ultra dan kredit foto habis, apakah harus membayar biaya training AI lagi? dan bagaimana cara generate foto lagi.<think>Kredit foto habis karena sudah digunakan 100 credits untuk paket Ultra. Biaya training AI sebesar Rp16.000 hanya dibayar sekali di awal dan tidak perlu lagi dibayar setelahnya. Untuk generate foto lagi, pengguna perlu melakukan additional payment (pembayaran tambahan) sebesar Rp10.000 per 10 credits. Karena 100 credits = Rp100.000, maka harga per foto (1 credit) menjadi Rp1.000. Semakin besar pak

Evaluating: 100%|██████████| 1/1 [00:43<00:00, 43.09s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'jika saya membeli paket ultra, lalu kredit sudah habis. apakah saya harus membayar biaya training AI lagi? lalu bagaimana caranya supaya bisa generate foto lagi?',
   'output': 'Mari kita pikirkan langkah demi langkah. 1. Pertanyaan menanyakan apakah harus membayar biaya pelatihan AI lagi jika sudah membeli paket Ultra tetapi kredit sudah habis, dan bagaimana cara agar bisa mengenerate foto lagi. 2. Berdasarkan informasi, biaya pelatihan AI sebesar Rp16.000 hanya berlaku untuk paket Facencial Ultra dan dikenakan sekali di awal untuk menyesuaikan model dengan foto pengguna. 3. Biaya pelatihan ini merupakan proses teknis yang wajib untuk paket Ultra, sehingga tidak bisa dihindari jika memilih paket tersebut. 4. Karena biaya pelatihan dibayar sekali di awal, jika kredits sudah habis pengguna tidak perlu membayar biaya pelatihan lagi untuk melanjutkan — cukup melakukan pembelian tambahan kredit (top-up) agar dapat mengenerate foto lagi. 5. Alternatif: jika ingin me

In [ ]:
evaluate_accuracy(['Saya sudah membeli Facencial Ultra, membayar Rp16.000 untuk training AI + 32 credits. Setelah upload 7 foto berkualitas tinggi (resolusi 4K, pencahayaan studio), hasilnya tidak mirip sama sekali - bahkan seperti wajah orang lain! Sudah coba generate 5x tetap buruk. Apakah ini kesalahan sistem? Bisakah saya minta refund Rp16.000 + pengembalian credits? Jika tidak, apakah tim support bisa membantu reproses training tanpa biaya tambahan? Bagaimana mekanisme komplain resminya?'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Saya sudah membeli Facencial Ultra, membayar Rp16.000 untuk training AI + 32 credits. Setelah upload 7 foto berkualitas tinggi (resolusi 4K, pencahayaan studio), hasilnya tidak mirip sama sekali - bahkan seperti wajah orang lain! Sudah coba generate 5x tetap buruk. Apakah ini kesalahan sistem? Bisakah saya minta refund Rp16.000 + pengembalian credits? Jika tidak, apakah tim support bisa membantu reproses training tanpa biaya tambahan? Bagaimana mekanisme komplain resminya?<think>Mari kita pikirkan langkah demi langkah. Facencial Ultra memerlukan 16.000 credit untuk training AI + 32 credits. Biaya pelatihan sebesar Rp16.000 dibayar sekali di awal; setelah training berhasil, credit awal akan dikurangi sebesar 32 credits dan pengguna akan mendapatkan 32 credits

Evaluating: 100%|██████████| 1/1 [01:06<00:00, 66.08s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'Saya sudah membeli Facencial Ultra, membayar Rp16.000 untuk training AI + 32 credits. Setelah upload 7 foto berkualitas tinggi (resolusi 4K, pencahayaan studio), hasilnya tidak mirip sama sekali - bahkan seperti wajah orang lain! Sudah coba generate 5x tetap buruk. Apakah ini kesalahan sistem? Bisakah saya minta refund Rp16.000 + pengembalian credits? Jika tidak, apakah tim support bisa membantu reproses training tanpa biaya tambahan? Bagaimana mekanisme komplain resminya?',
   'output': 'Mari kita pikirkan langkah demi langkah. Pengguna menanyakan apakah Facencial memberikan pengembalian dana atau pengembalian credits jika hasil training/generate tidak memuaskan, dan apakah ada opsi reproses tanpa biaya. Berdasarkan informasi kebijakan yang tersedia: 1) Facencial menggunakan teknologi AI dengan biaya komputasi yang tinggi, sehingga secara umum pengembalian dana (refund) tidak diberikan. 2) Tidak ada garansi uang kembali jika hasil foto tidak sepenuhnya menyeru

In [ ]:
evaluate_accuracy(["Setelah pakai Facencial Lite untuk 20 foto kampus, hasilnya kurang halus. Saya pertimbangkan upgrade ke Ultra. Tapi: 1) Apakah foto yang sudah di-generate di Lite bisa di-reprocess di Ultra tanpa upload ulang? 2) Jika harus training baru, apakah biaya Rp16.000 berlaku lagi? 3) Untuk efisiensi, apakah bisa pakai 1 model Ultra untuk generate semua gaya (formal, olahraga, seni)? 4) Berapa total biaya paling optimal untuk 50 foto kualitas premium?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Setelah pakai Facencial Lite untuk 20 foto kampus, hasilnya kurang halus. Saya pertimbangkan upgrade ke Ultra. Tapi: 1) Apakah foto yang sudah di-generate di Lite bisa di-reprocess di Ultra tanpa upload ulang? 2) Jika harus training baru, apakah biaya Rp16.000 berlaku lagi? 3) Untuk efisiensi, apakah bisa pakai 1 model Ultra untuk generate semua gaya (formal, olahraga, seni)? 4) Berapa total biaya paling optimal untuk 50 foto kualitas premium?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan pengguna berisi empat poin: 1) apakah foto dari Lite bisa di-reprocess di Ultra tanpa upload ulang, 2) jika harus training baru apakah biaya Rp16.000 berlaku lagi, 3) apakah bisa pakai 1 model Ultra untuk semua gaya, dan 4) total biaya optimal untuk 50 foto prem

Evaluating: 100%|██████████| 1/1 [01:17<00:00, 77.31s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'Setelah pakai Facencial Lite untuk 20 foto kampus, hasilnya kurang halus. Saya pertimbangkan upgrade ke Ultra. Tapi: 1) Apakah foto yang sudah di-generate di Lite bisa di-reprocess di Ultra tanpa upload ulang? 2) Jika harus training baru, apakah biaya Rp16.000 berlaku lagi? 3) Untuk efisiensi, apakah bisa pakai 1 model Ultra untuk generate semua gaya (formal, olahraga, seni)? 4) Berapa total biaya paling optimal untuk 50 foto kualitas premium?',
   'output': 'Mari kita pikirkan langkah demi langkah. Berdasarkan informasi retrieval: Facencial Ultra membutuhkan proses pelatihan AI terlebih dahulu yang dibayar di awal sebesar Rp16.000; setelah pelatihan, pengguna dapat mengenerate foto menggunakan credit tanpa perlu pelatihan ulang. Facencial Lite tidak melalui proses pelatihan tersebut sehingga tidak dikenai biaya pelatihan.\n\n<output>1) Apakah foto yang sudah di-generate di Lite bisa di-reprocess di Ultra tanpa upload ulang?\nBerdasarkan informasi retrieval, Fa

In [ ]:
evaluate_accuracy(["Sebagai fotografer profesional, saya tertarik integrasikan Facencial ke studio foto kami. Apakah tersedia API untuk otomasi generate foto pelanggan? Jika ya, bagaimana skema biayanya (per hit API atau paket khusus)? Apakah ada fitur watermark khusus studio? Juga, untuk kebutuhan cetak fisik, resolusi maksimal hasil foto berapa DPI? Apakah mendukung format CMYK untuk printing profesional?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Sebagai fotografer profesional, saya tertarik integrasikan Facencial ke studio foto kami. Apakah tersedia API untuk otomasi generate foto pelanggan? Jika ya, bagaimana skema biayanya (per hit API atau paket khusus)? Apakah ada fitur watermark khusus studio? Juga, untuk kebutuhan cetak fisik, resolusi maksimal hasil foto berapa DPI? Apakah mendukung format CMYK untuk printing profesional?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan mencakup beberapa hal: tersedia API, skema biaya, fitur watermark, resolusi untuk printing, dan dukungan format. Karena Facencial berbasis AI dan hasilnya disimpan di model AI tanpa perlu proses rendering ulang, maka biaya awal AI (Uang Kertas) tidak dikenakan lagi untuk generate foto. Namun, ada biaya pelatihan AI ta

Evaluating: 100%|██████████| 1/1 [00:49<00:00, 49.07s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'Sebagai fotografer profesional, saya tertarik integrasikan Facencial ke studio foto kami. Apakah tersedia API untuk otomasi generate foto pelanggan? Jika ya, bagaimana skema biayanya (per hit API atau paket khusus)? Apakah ada fitur watermark khusus studio? Juga, untuk kebutuhan cetak fisik, resolusi maksimal hasil foto berapa DPI? Apakah mendukung format CMYK untuk printing profesional?',
   'output': 'Mari kita pikirkan langkah demi langkah. Berdasarkan informasi yang tersedia dari hasil retrieval, hanya beberapa bagian dari pertanyaan Anda yang dapat dijawab dengan pasti: \n\n1) Cara kerja AI: Facencial menggunakan teknologi generatif AI yang menerima foto input dari pengguna, menganalisis fitur wajah, ekspresi, dan komposisi, lalu menghasilkan variasi foto profesional dengan gaya dan latar belakang berbeda.\n\n2) Garansi / refund: Karena biaya komputasi yang cukup besar, Facencial tidak memberikan refund. Jika pengguna tidak puas dengan hasil, mereka dapat 

In [ ]:
evaluate_accuracy(["saya kurang puas nih dengan hasilnya. bintang satu!"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>saya kurang puas nih dengan hasilnya. bintang satu!<think>Mari kita pikirkan langkah demi langkah.
1. Pengguna merasa kurang puas dengan hasil foto dan memberi bintang satu.
2. Facencial Lite menghasilkan foto tanpa proses pelatihan AI, sehingga untuk hasil terbaik disarankan menggunakan Facencial Ultra.
3. Proses pelatihan AI membutuhkan biaya Rp16.000 di awal, tapi setelah itu pengguna bisa generate foto sebanyak yang diinginkan menggunakan credit.
4. Dengan membeli paket pelatihan ini, pengguna menghindari biaya per foto yang tinggi (Rp10.000/foto) dan mendapatkan hasil foto yang lebih realistis dan sesuai ekspektasi.
5. Oleh karena itu, rekomendasi terbaik adalah membeli paket pelatihan AI untuk hasil yang lebih memuaskan dan menghindari hasil yang kuran

Evaluating: 100%|██████████| 1/1 [00:40<00:00, 40.04s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'saya kurang puas nih dengan hasilnya. bintang satu!',
   'output': 'Mari kita pikirkan langkah demi langkah.\n1. Anda merasa kurang puas dengan hasil foto dan memberi bintang satu.\n2. Facencial Lite menghasilkan foto tanpa proses pelatihan AI, sehingga hasilnya bisa bervariasi tergantung kualitas foto input, pencahayaan, ekspresi, dan gaya yang dipilih.\n3. Facencial Ultra melakukan pelatihan AI terlebih dahulu (biaya awal Rp16.000), sehingga model dapat menyesuaikan lebih baik dengan fitur Anda; setelah pelatihan Anda bisa generate foto menggunakan credit.\n4. Karena biaya komputasi AI yang tinggi, Facencial tidak menyediakan refund.\n5. Jika hasil tidak memuaskan, silakan hubungi support@facencial.com — tim support dapat membantu menganalisis penyebab (mis. kualitas input atau gaya kurang cocok) dan memberi panduan seperti mengganti foto input atau memilih gaya berbeda.\n\n<output>Kami mengerti Anda kecewa. Karena biaya komputasi yang tinggi kami tidak membe

In [ ]:
evaluate_accuracy(["apakah facencial menyediakan jasa perawatan wajah? karena dari namanya terdapat 'face' sehingga saya prediksi ada layanan perawatan wajah"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah facencial menyediakan jasa perawatan wajah? karena dari namanya terdapat 'face' sehingga saya prediksi ada layanan perawatan wajah<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah Facencial menyediakan jasa perawatan wajah dan mengaitkan dengan adanya kata 'face' pada namanya. Berdasarkan informasi yang tersedia, Facencial adalah layanan AI generatif untuk membuat foto, bukan layanan perawatan wajah langsung. Meskipun namanya mengandung 'face', tidak ada indikasi bahwa Facencial menyediakan perawatan wajah seperti proses foto atau editing fisik.<think><output>Facencial tidak menyediakan jasa perawatan wajah; namanya berasal dari kata 'face' yang merujuk pada layanan AI generatif untuk membuat foto.</output><｜end▁of▁sentence｜

Evaluating: 100%|██████████| 1/1 [00:19<00:00, 19.34s/it]


✅ Akurasi: 100.00% (1/1 benar)


(1.0, [])

In [ ]:
evaluate_accuracy(["apakah perlu upload KTP dulu untuk daftar/beli facencial?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah perlu upload KTP dulu untuk daftar/beli facencial?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah pengguna perlu mengunggah dokumen KTP sebagai bagian dari proses registrasi atau pembelian layanan Facencial. Berdasarkan informasi yang tersedia, tidak ada indikasi bahwa pengunggahan KTP wajib dilakukan pada tahapan awal proses tersebut. KTP tidak tercantum sebagai dokumen input yang harus disediakan pengguna dalam bagian persyaratan umum atau panduan setup akun. Karena itu, pertanyaan pengguna tentang kebutuhan upload KTP tidak didukung oleh informasi yang tersedia, sehingga jawaban yang paling akurat adalah bahwa tidak perlu mengunggah KTP di awal.<think><output>Tidak perlu upload KTP dulu untuk daftar/beli Facencial. Sila

Evaluating: 100%|██████████| 1/1 [00:41<00:00, 41.57s/it]


✅ Akurasi: 100.00% (1/1 benar)


(1.0, [])

In [ ]:
evaluate_accuracy(["bagaimana facencial menjaga keamanan privasi foto-foto pengguna?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>bagaimana facencial menjaga keamanan privasi foto-foto pengguna?<think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan prosedur yang digunakan oleh Facencial untuk memastikan keamanan dan kerahasiaan foto pengguna.
2. Facencial menerapkan berbagai teknik untuk menjaga keamanan, termasuk menghapus foto yang telah diproses secara otomatis setelah upload.
3. Proses ini dilakukan untuk menghindari penyimpanan data yang tidak perlu dan meminimalisir risiko kebocoran.
4. Oleh karena itu, pengguna dapat merasa lebih aman menggunakan layanan Facencial karena adanya langkah-langkah pencegahan keamanan ini.<think><output>Facencial menjaga keamanan privasi foto-foto pengguna dengan menghapus foto yang telah diproses secara otomatis setelah upload. Sem

Evaluating: 100%|██████████| 1/1 [00:36<00:00, 36.27s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'bagaimana facencial menjaga keamanan privasi foto-foto pengguna?',
   'output': 'Mari kita pikirkan langkah demi langkah.\n1. Pertanyaan menanyakan bagaimana Facencial menjaga keamanan dan privasi foto pengguna.\n2. Berdasarkan informasi yang tersedia, Facencial menerima foto input dan menggunakan teknologi generatif AI untuk menganalisis fitur wajah dan menghasilkan variasi foto profesional.\n3. Juga disebutkan bahwa Facencial memiliki sistem AI untuk validasi gambar sebelum upload agar kualitas input memadai.\n4. Namun, dalam sumber yang disediakan tidak ada keterangan spesifik mengenai langkah-langkah keamanan atau privasi (misalnya enkripsi, kebijakan penyimpanan, atau penghapusan otomatis file).\n5. Oleh karena itu, tidak dapat ditarik kesimpulan tentang mekanisme perlindungan privasi dari foto pengguna berdasarkan data yang ada; untuk informasi detail, pengguna disarankan menghubungi tim support.\n<output>Berdasarkan informasi yang tersedia, Facencial mem

In [ ]:
evaluate_accuracy(["apa yang bisa saya dapatkan dengan facencial lite?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apa yang bisa saya dapatkan dengan facencial lite?<think>Mari kita pikirkan langkah demi langkah. Facencial Lite adalah versi dasar dari layanan AI generatif Facencial. Versi ini tetap dapat menghasilkan foto berkualitas tinggi, tetapi dengan beberapa batasan: pengguna tidak bisa mengubah gaya foto, latar belakang, atau warna mata karakter—pengguna juga tidak perlu menunggu proses pelatihan AI (training) sebelumnya. Karena batasan ini, Facencial Lite tidak memerlukan biaya pelatihan sebesar Rp16.000 di awal. Akibatnya, pengguna dapat menghasilkan foto lebih cepat dan lebih murah dibandingkan Facencial Ultra.<think><output>Facencial Lite memungkinkan Anda menghasilkan foto berkualitas tinggi dalam waktu 10 detik dengan harga mulai Rp10.000. Namun, Anda tidak 

Evaluating: 100%|██████████| 1/1 [00:41<00:00, 41.72s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'apa yang bisa saya dapatkan dengan facencial lite?',
   'output': 'Mari kita pikirkan langkah demi langkah. Facencial Lite adalah versi dasar yang tidak melewati proses pelatihan AI terlebih dahulu, sehingga pengguna tidak dikenai biaya pelatihan dan proses pembuatan foto menjadi lebih cepat dan lebih murah. Karena tidak ada pelatihan khusus, kualitas foto yang dihasilkan tetap realistis tetapi cenderung kurang detail atau kurang mirip dibandingkan Facencial Ultra yang telah dilatih. Facencial Lite juga memiliki batasan fitur: pengguna tidak dapat mengubah gaya foto, latar belakang, atau warna mata karakter. Untuk memulai dengan Facencial Lite, pengguna hanya perlu mengunggah 1 foto wajah; sistem akan menganalisis gender dan atribut wajah, lalu Anda dapat mengenerate foto menggunakan gaya/template yang tersedia.\n<output>Facencial Lite memungkinkan Anda menghasilkan foto realistis tanpa melalui proses pelatihan AI (jadi tidak ada biaya pelatihan tambahan), sehi

In [ ]:
evaluate_accuracy(["apakah ada diskon?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah ada diskon?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah ada diskon yang tersedia untuk layanan Facencial. Dari informasi yang tersedia, diskon dapat diperoleh melalui paket credit yang lebih besar atau dengan menggunakan credit Facencial. Karena itu, jawaban yang benar adalah ada diskon, dan penjelasannya sesuai dengan yang berada di pikiran.<think><output>Ya, ada diskon jika membeli paket credit besar atau menggunakan credit Facencial.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 1/1 [00:25<00:00, 25.39s/it]


✅ Akurasi: 100.00% (1/1 benar)


(1.0, [])

In [ ]:
evaluate_accuracy(["apakah ada layanan yang free? "])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah ada layanan yang free? <think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan apakah ada layanan gratis di Facencial.
2. Facencial Ultra menggunakan biaya pelatihan AI Rp16.000 di awal, tapi setelah itu pengguna bisa generate foto sebanyak yang diinginkan menggunakan credit.
3. Satu credit setara dengan satu foto, dan credit tidak bisa didapatkan secara gratis.
4. Satu foto 1000x1000 memiliki harga Rp100, sehingga untuk mendapatkan 1 foto gratis, harga per foto harus dibawah Rp100.
5. Tidak ada informasi bahwa harga awal Rp16.000 atau biaya pelatihan AI bisa dibayar ulang setiap hari, sehingga tidak ada layanan yang benar-benar gratis secara berulang.<think><output>Facencial tidak menyediakan layanan yang gratis secara berulang. Kamu

Evaluating: 100%|██████████| 1/1 [00:49<00:00, 49.10s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'apakah ada layanan yang free? ',
   'output': 'Mari kita pikirkan langkah demi langkah. 1) Pengguna menanyakan apakah ada layanan gratis di Facencial. 2) Berdasarkan informasi pricing, Facencial tidak menyediakan layanan gratis atau credits gratis. 3) Paket termurah adalah Realistic Lite, mulai dari Rp10.000 untuk 16 credits (sekitar Rp625 per foto). 4) Untuk paket Ultra Realistic ada biaya pelatihan AI sekali bayar sebesar Rp16.000 yang tidak bisa dihindari jika memilih paket tersebut. 5) Semua biaya tercantum secara transparan di halaman pricing, sehingga tidak ada biaya tersembunyi.\n<output>Tidak, tidak ada layanan gratis. Paket termurah Realistic Lite mulai dari Rp10.000 untuk 16 credits (≈ Rp625/foto). Untuk Ultra Realistic ada biaya pelatihan AI sekali sebesar Rp16.000. Semua biaya tercantum di halaman pricing dan credits tidak diberikan secara gratis.</output>'}])

In [ ]:
evaluate_accuracy(["apakah ada email bantuan?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah ada email bantuan?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah ada layanan email bantuan untuk pengguna yang membutuhkan dukungan. Dari informasi yang tersedia, Facencial tidak menyediakan alamat email khusus untuk bantuan; pengguna harus menggunakan fitur support@facencial.com yang tersedia 24/7 melalui situs web atau aplikasi.<think><output>Kami memahami kebutuhan Anda, namun kami tidak menyediakan layanan email bantuan terlebih dahulu. Silakan hubungi support@facencial.com untuk mendapatkan bantuan.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 1/1 [00:25<00:00, 25.67s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'apakah ada email bantuan?',
   'output': 'Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah ada alamat email untuk bantuan. Berdasarkan informasi yang tersedia, Facencial menyediakan dukungan pelanggan melalui alamat email support@facencial.com. Email ini dapat dihubungi untuk mendapatkan bantuan, termasuk jika Anda tidak puas dengan hasil; tim support akan membantu mencari solusi. Layanan ini tersedia melalui situs web atau aplikasi, dan dapat dihubungi kapan pun.\n\n<output>Ya — silakan hubungi tim dukungan kami di support@facencial.com untuk bantuan.</output>'}])

In [ ]:
evaluate_accuracy(["apakah saya mendapatkan foto fisiknya?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah saya mendapatkan foto fisiknya?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah pengguna dapat mendapatkan foto fisik. Informasi yang tersedia hanya menyebutkan bahwa pengguna dapat mengunduh foto dalam format digital melalui halaman hasil. Tidak ada informasi bahwa layanan menyediakan foto fisik.<think><output>Facencial tidak menyediakan foto fisik. Anda dapat mengunduh foto dalam format digital melalui halaman hasil.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 1/1 [00:20<00:00, 20.84s/it]


✅ Akurasi: 100.00% (1/1 benar)


(1.0, [])

In [ ]:
evaluate_accuracy(["saya sudah membeli paket. akan tetapi ketika klik tombol generate, gambar tidak muncul"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>saya sudah membeli paket. akan tetapi ketika klik tombol generate, gambar tidak muncul<think>Mari kita pikirkan langkah demi langkah. Pengguna telah membeli paket layanan Facencial, namun pada saat proses generate, hasilnya tidak muncul atau error. Kode error yang muncul adalah 400 Bad Request, yang berarti ada permintaan yang salah dari pengguna. Kode ini biasanya terjadi karena beberapa alasan: data input tidak valid, ukuran gambar yang diupload terlalu besar, atau atribut wajah yang dipilih tidak didukung.<think><output>Kode error 400 Bad Request berarti ada yang salah dengan input Anda. Pastikan Anda mengupload foto dengan ukuran maksimal 512x512 pixels, lalu klik 'GENERATE SEKARANG' dan tunggu sebentar.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 1/1 [00:32<00:00, 32.10s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'saya sudah membeli paket. akan tetapi ketika klik tombol generate, gambar tidak muncul',
   'output': "Mari kita pikirkan langkah demi langkah. 1. Pengguna sudah membeli paket namun gambar tidak muncul kemungkinan karena proses generate belum dijalankan sesuai prosedur. 2. Untuk memulai pembuatan foto, pastikan Anda sudah login ke akun Facencial. 3. Setelah login, pastikan paket atau credits yang Anda beli sudah dipilih/aktif. 4. Unggah foto yang akan diproses dan pilih gaya yang diinginkan. 5. Klik tombol 'GENERATE SEKARANG' dan tunggu sampai proses selesai dan gambar muncul.\n<output>Pastikan Anda sudah login, memilih paket/credits yang dibeli, lalu unggah foto dan pilih gaya. Setelah itu klik 'GENERATE SEKARANG' dan tunggu hingga gambar muncul.</output>"}])

# Fine-tuning phase III


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
lora_rank = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "MMumtazSakho/Facencial-CoT-Lora-updated",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-10 02:12:53 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-10 02:12:53 [__init__.py:239] Automatically detected platform cuda.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.4: Fast Qwen3 patching. Transformers: 4.55.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.78 GB. Also swap space = 0 GB.


...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/175M [00:00<?, ?B/s]

Unsloth 2025.8.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [4]:
reasoning_start = "<think>" # Acts as <think>
reasoning_end   = "<think>"   # Acts as </think>
solution_start  = "<output>"
solution_end    = "</output>"

system_prompt = \
f"""Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada {reasoning_start} dan {reasoning_end}.
lalu, letakkan jawaban kamu di antara {solution_start}{solution_end}"""
system_prompt

'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output>'

In [5]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

In [6]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset_updated = load_dataset("MMumtazSakho/facencial-cot-updated", split = "train")
dataset_updated = dataset_updated.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

dataset_updated

dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/52 [00:00<?, ? examples/s]

,expected_answer,problem,generated_solution
0,Facencial menggunakan teknologi generatif AI t...,Apa keunggulan utama Facencial?,Mari kita pikirkan langkah demi langkah.\n1. P...
1,"Klik tombol 'GENERATE SEKARANG' untuk login, p...",Bagaimana cara mulai generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...
2,Realistic Lite memiliki kualitas basic dengan ...,Apa perbedaan Realistic Lite dan Ultra Realistic?,Mari kita pikirkan langkah demi langkah.\n1. P...
3,Paket termurah adalah Realistic Lite mulai dar...,Berapa harga paket termurah?,Mari kita pikirkan langkah demi langkah.\n1. P...
4,Proses generate foto memakan waktu 20 detik un...,Berapa lama proses generate foto?,Mari kita pikirkan langkah demi langkah.\n1. P...
5,Edisi Kampus memungkinkan Anda berfoto AI di l...,Apa itu Edisi Kampus?,Mari kita pikirkan langkah demi langkah.\n1. P...
6,Kami memahami kebutuhan Anda dan berupaya memb...,Apakah ada garansi kepuasan?,Mari kita pikirkan langkah demi langkah.\n1. P...
7,Paket 100 credits (Paket D) dijual seharga Rp5...,Berapa harga paket 100 credits?,Mari kita pikirkan langkah demi langkah.\n1. P...
8,"Ya, Facencial memiliki sistem AI validasi gamb...",Apakah ada validasi gambar sebelum upload?,Mari kita pikirkan langkah demi langkah.\n1. P...
9,Formal Pas Foto untuk kebutuhan profesional de...,Apa perbedaan Formal Pas Foto dan Foto Seni?,Mari kita pikirkan langkah demi langkah.\n1. P...


In [7]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    thoughts = thoughts.strip()
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset_updated["Messages"] = dataset_updated.apply(format_dataset, axis = 1)

In [8]:
tokenizer.apply_chat_template(dataset_updated["Messages"][44], tokenize = False)

'Kamu adalah chatbot costumer service Facencial.\npikirkan jawaban dari pertanyaan user sebelum kamu menjawab.\nletakkan pikiran kamu pada <think> dan <think>.\nlalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Berapa lama model AI saya tersimpan? apakah saya bisa menghapusnya? jika terhapus, maka apa akibatnya?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan mencakup tiga hal: durasi penyimpanan, opsi penghapusan, dan konsekuensi jika dihapus. Dari informasi yang tersedia, yang pasti adalah: model dapat dihapus lewat Pengaturan Akun, dan bila dihapus lalu Anda ingin menggunakan Ultra Realistic lagi, sistem harus melatih ulang dari awal dan biaya pelatihan Rp16.000 akan dikenakan kembali. Durasi penyimpanan spesifik tidak tercantum pada informasi yang saya miliki, sehingga sebaiknya merujuk ke kebijakan/halaman bantuan resmi untuk detail terbarunya.<think><output>• Durasi penyimpanan: Tidak tercantum pada info yang saya miliki. Silakan cek Pusat Bantua

In [9]:
dataset_updated["N"] = dataset_updated["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset_updated = dataset_updated.loc[dataset_updated["N"] <= max_seq_length/2].copy()
dataset_updated.shape

(52, 5)

In [10]:
from datasets import Dataset

dataset_updated["text"] = tokenizer.apply_chat_template(dataset_updated["Messages"].values.tolist(), tokenize = False)
dataset_updated = Dataset.from_pandas(dataset_updated)
dataset_updated

Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'Messages', 'N', 'text', '__index_level_0__'],
    num_rows: 52
})

In [11]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_updated,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/52 [00:00<?, ? examples/s]

In [12]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 52 | Num Epochs = 5 | Total steps = 260
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)


Step,Training Loss
5,0.181600
10,0.043400
15,0.699900
20,0.076800
25,0.265300
30,0.536600
35,0.168700
40,0.107600
45,0.293900
50,0.095600


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=260, training_loss=0.10105429962277412, metrics={'train_runtime': 373.3571, 'train_samples_per_second': 0.696, 'train_steps_per_second': 0.696, 'total_flos': 2209778438768640.0, 'train_loss': 0.10105429962277412})

In [19]:
evaluate_accuracy(["Apakah facencial menyediakan API untuk integrasi generate foto?"])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Apakah facencial menyediakan API untuk integrasi generate foto?<think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan apakah Facencial menyediakan API untuk mengintegrasikan layanan generate foto ke aplikasi atau platform lain.
2. Facencial menggunakan teknologi AI yang memerlukan proses komputasi yang intensif, sehingga tidak tersedia melalui API publik.
3. Untuk kebutuhan integrasi, pengguna harus menggunakan layanan Facencial melalui situs web atau aplikasi mobile resmi.
4. Alternatif yang dapat dipertimbangkan adalah menggunakan layanan komputasi awan (cloud computing) yang terintegrasi dengan Facencial, tetapi ini tetap memerlukan proses training AI terlebih dahulu.
5. Dengan demikian, saat ini tidak ada cara yang resmi untuk menginteg

Evaluating: 100%|██████████| 1/1 [01:01<00:00, 61.04s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'Apakah facencial menyediakan API untuk integrasi generate foto?',
   'output': 'Mari kita pikirkan langkah demi langkah. 1. Pertanyaan menanyakan apakah Facencial menyediakan API untuk integrasi layanan generate foto. 2. Informasi yang tersedia menjelaskan bahwa Facencial adalah layanan AI generatif untuk membuat foto, bagaimana AI bekerja, dan kebijakan terkait (mis. mengenai biaya komputasi dan dukungan pelanggan). 3. Namun, dalam sumber yang tersedia tidak ada pernyataan eksplisit yang menyebutkan ketersediaan API publik atau mekanisme integrasi resmi. 4. Karena itu, berdasarkan informasi yang ada saat ini, kita tidak dapat memastikan apakah Facencial menyediakan API untuk integrasi. 5. Untuk memperoleh konfirmasi dan informasi mengenai opsi integrasi, silakan menghubungi tim dukungan Facencial di support@facencial.com.\n\n<output>Menurut informasi yang tersedia, tidak ada keterangan eksplisit mengenai API publik untuk integrasi. Untuk mendapatkan konfirmasi

# Model Evaluation Phase III


In [18]:
test_case = ['Apakah harga per foto di Ultra Realistic lebih mahal daripada Lite?',
 'Apa syarat foto yang bisa diupload ke Facencial Ultra?',
 'Mengapa Facencial Ultra lebih lama prosesnya daripada Lite?',
 'Mana yang lebih cocok untuk dokumen resmi: Formal Pas Foto atau Foto Seni?',
 'Apa yang harus dilakukan jika proses training Ultra gagal?',
 'Facencial Ultra lebih bagus dari Lite, ya?',
 'Credit habis tapi mau generate ulang, gimana?',
 'Berapa harga Paket D?',
 'Berapa harga Paket A?',
 'Ada berapa foto yang saya dapatkan di paket D?',
 'Ada berapa foto yang saya dapatkan di paket B?',
 'Apakah biaya pelatihan AI berlaku untuk semua paket?',
 'Paket mana yang paling hemat untuk kebutuhan foto banyak?',
 'Berapa total biaya awal untuk Facencial Ultra termasuk pelatihan AI?',
 'Kalau wajahnya pakai kacamata, apakah bisa terdeteksi?',
 'saya sudah pernah membeli paket ultra. tetapi credit sudah habis. apakah harus membayar biaya pelatihan lagi?',
 'berapa lama prosesnya?',
 'apakah jumlah credit bisa custom?',
 'siapa nama presiden indonesia?',
 'berapa nilai 1+1?',
 'bagaimana cara memasak?',
 'Berapa lama model AI saya tersimpan? apakah saya bisa menghapusnya? jika terhapus, maka apa akibatnya?',
  'saya frustasi dengan hasil foto yang tidak konsisten',
  'adakah kontak whatsapp yang dapat dihubungi? saya mau bertanya',
  'jika saya membeli paket ultra, lalu kredit sudah habis. apakah saya harus membayar biaya training AI lagi? lalu bagaimana caranya supaya bisa generate foto lagi?',
 'saya kurang puas nih dengan hasilnya. bintang satu!',
'apakah ada layanan yang free?',
'apakah ada email bantuan?'
  ]

In [20]:
evaluate_accuracy(test_case)

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Apakah harga per foto di Ultra Realistic lebih mahal daripada Lite?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah harga per foto Ultra Realistic lebih mahal daripada Lite. Dari informasi yang tersedia, perbedaan yang disebut adalah adanya biaya pelatihan AI Rp16.000 sekali di awal untuk Ultra Realistic, sedangkan Lite tidak memiliki biaya ini. Tidak ada informasi yang menyatakan harga per foto (credit) Ultra lebih mahal daripada Lite, sehingga kesimpulan tersebut tidak bisa dipastikan dari data ini.<think><output>Tidak ada informasi bahwa harga per foto Ultra Realistic lebih mahal daripada Lite. Yang pasti, Ultra Realistic memiliki biaya pelatihan AI Rp16.000 sekali di awal, sedangkan Lite tidak. Untuk detail harga per foto/cred

Evaluating:   4%|▎         | 1/28 [00:29<13:23, 29.75s/it]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>Apa syarat foto yang bisa diupload ke Facencial Ultra?<think>Mari kita pikirkan langkah demi langkah.
1. Pertanyaan menanyakan syarat foto yang bisa diupload ke Facencial Ultra.
2. Facencial Ultra memerlukan kualitas foto yang tinggi untuk menghasilkan gambar dengan detail yang memadai.
3. Resolusi minimal foto adalah 512x512 piksel karena area aktif (aktive area) foto harus mencakup wajah pengguna.
4. Format file yang diperbolehkan adalah PNG atau JPG karena kedua format ini umum digunakan dan mendukung kompresi serta transparansi (walaupun untuk JPG tidak support transparansi, tapi resolusi dan kualitas tetap menjadi faktor utama).
5. Ukuran file maksimal 5MB diterapkan untuk mencegah loading yang lambat dan memastikan komputasi dapat dilakukan dengan baik

Evaluating:   7%|▋         | 2/28 [00:58<12:42, 29.33s/it]

...

Evaluating:  96%|█████████▋| 27/28 [11:13<00:28, 28.36s/it]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah ada email bantuan?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan menanyakan apakah ada alamat email untuk bantuan. Berdasarkan informasi yang tersedia, Facencial menyediakan dukungan pelanggan melalui alamat email support@facencial.com. Email ini dapat dihubungi untuk mendapatkan bantuan, termasuk jika Anda tidak puas dengan hasil; tim support akan membantu mencari solusi. Layanan ini tersedia melalui situs web atau aplikasi, dan dapat dihubungi kapan pun.<think><output>Ya — silakan hubungi tim dukungan kami di support@facencial.com untuk bantuan.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 28/28 [11:28<00:00, 24.60s/it]


✅ Akurasi: 60.71% (17/28 benar)


(0.6071428571428571,
 [{'instruction': 'Apa syarat foto yang bisa diupload ke Facencial Ultra?',
   'output': 'Mari kita pikirkan langkah demi langkah.\n1. Untuk menggunakan Facencial Ultra, kamu harus mengupload minimal 5 foto wajah sebagai data pelatihan. \n2. Setelah upload, sistem akan memverifikasi kualitas foto; hanya foto yang lolos verifikasi kualitas yang akan diproses. \n3. Setelah foto terverifikasi, AI kami akan melakukan pelatihan untuk memahami karakteristik wajah (proses ini memakan waktu sekitar 5 menit), lalu kamu dapat mengenerate foto dalam berbagai gaya.\n<output>Upload minimal 5 foto wajah dengan kualitas baik; setelah foto terverifikasi, sistem akan melakukan pelatihan (sekitar 5 menit) dan kemudian kamu bisa mengenerate foto.</output>'},
  {'instruction': 'Mengapa Facencial Ultra lebih lama prosesnya daripada Lite?',
   'output': 'Mari kita pikirkan langkah demi langkah. \n1. Pertanyaan menanyakan mengapa Facencial Ultra memerlukan waktu lebih lama dibandingkan F

## Model Evaluation Phase III
## ACC: 86%

Berdasarkan hasil koreksi RAG GPT-5, hasil finetuned model memiliki akurasi 60% (17/28). namun setelah di cek, terdapat kesalaan koreksi oleh RAG GPT-5. berdasarkan Feedback manusia, seharusnya chatbot ini benar 24/28 alias 86%

In [25]:
evaluate_accuracy(['berapa hasil perkalian dari 3x3?'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>berapa hasil perkalian dari 3x3?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan berapa hasil perkalian dari 3x3 berada di luar konteks layanan Facencial, Sehingga saya tidak diperkenankan menjawab pertanyaan tersebut.<think><output>Maaf, Saya adalah Assistant CS Facencial. Saya tidak dapat menjawab pertanyaan tersebut. Silakan bertanya mengenai Facencial.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 1/1 [00:26<00:00, 26.67s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'berapa hasil perkalian dari 3x3?',
   'output': 'Mari kita pikirkan langkah demi langkah. Pertanyaan tentang hasil perkalian 3x3 berada di luar konteks layanan Facencial. Sebagai Assistant CS Facencial, fokus jawaban saya adalah seputar produk dan layanan Facencial (mis. credits, paket, kualitas foto, diskon, proses generate foto). Untuk pertanyaan di luar domain tersebut, saya tidak memberikan jawaban matematika dan akan meminta pengguna untuk menanyakan hal yang berkaitan dengan Facencial.\n<output>Maaf, saya adalah Assistant CS Facencial. Saya tidak dapat menjawab pertanyaan tersebut. Silakan bertanya mengenai Facencial.</output>'}])

In [27]:
evaluate_accuracy(['apakah saya tampan?'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Kamu adalah chatbot costumer service Facencial.
pikirkan jawaban dari pertanyaan user sebelum kamu menjawab.
letakkan pikiran kamu pada <think> dan </think>.
lalu, letakkan jawaban kamu di antara <output></output><｜end▁of▁sentence｜>apakah saya tampan?<think>Mari kita pikirkan langkah demi langkah. Pertanyaan "apakah saya tampan?" bersifat subjektif dan bergantung pada preferensi personal. Namun, sebagai bagian dari layanan Facencial, saya tidak diperkenankan untuk membuat pernyataan yang bersifat opini atau value-based. Karena itu, saya tidak bisa menjawab pertanyaan tersebut.<think><output>Maaf, Saya adalah Assistant CS Facencial. Saya tidak diperkenankan menjawab pertanyaan tersebut. Silakan bertanya mengenai produk atau layanan Facencial.</output><｜end▁of▁sentence｜>


Evaluating: 100%|██████████| 1/1 [00:36<00:00, 36.71s/it]


✅ Akurasi: 0.00% (0/1 benar)


(0.0,
 [{'instruction': 'apakah saya tampan?',
   'output': 'Mari kita pikirkan langkah demi langkah. 1. Pertanyaan "apakah saya tampan?" bersifat subjektif dan berada di luar konteks produk/layanan Facencial. 2. Sebagai layanan customer service Facencial, fokus jawaban adalah pada informasi mengenai produk, layanan, dan dukungan purna jual. 3. Jika Anda ingin penilaian terkait tampilan dalam foto, Facencial menyediakan produk Formal Pas Foto yang dirancang untuk kebutuhan resmi termasuk paspor. 4. Facencial memiliki rating 5/5 untuk kualitas foto dan fleksibilitas gaya. 5. Jika Anda tidak puas dengan hasil, silakan menghubungi tim support karena kebijakan refund terbatas.\n<output>Maaf, saya adalah Assistant CS Facencial dan saya tidak dapat menilai apakah Anda tampan. Silakan tanyakan mengenai produk atau layanan Facencial—misalnya Formal Pas Foto (cocok untuk paspor), informasi kualitas foto (Facencial memiliki rating 5/5), atau bantuan purna jual melalui support@facencial.com.</out

In [ ]:
tokenizer.push_to_hub("MMumtazSakho/Facencial-CoT-Lora-updated-v2", token = "hf-token")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpd0b920ql/tokenizer.json       : 100%|##########| 11.4MB / 11.4MB            

In [ ]:
model.push_to_hub("MMumtazSakho/Facencial-CoT-Lora-updated-v2", token = "hf-token")

README.md: 0.00B [00:00, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pvahaeppw/adapter_model.safetensors:   0%|          |  558kB /  175MB            

Saved model to https://huggingface.co/MMumtazSakho/Facencial-CoT-Lora-updated-v2


# Summary

1. CoT (chain of thought) is very helpful to make the model more understand the context
2. Model accuracy increasing as the model being thaught by expert
3. the final model accuracy is 86% in 30 testing data